# คำนวณ MFB และสถิติอื่น ๆ ของทุกเรดาร์ ด้วยการเทียบกับฝนสถานี เพื่อนำไปปรับแก้ bias แต่ละสถานีก่อนไปโมเสค
* จะได้ผลลัพธ์ค่า MFB แต่ละสถานีเรดาร์ ที่คำนวณโดยใช้ฝนสถานี สามารถนำเอา MFB ไปใช้ปรับแก้เพื่อลดค่าเอนเอียงได้

In [1]:
'''
2024.09.22
โค้ดนี้พัฒนาโดย รองศาสตราจารย์ ดร. นัฐพล มหาวิค ภาควิชาทรัพยากรธรรมชาติและสิ่งแวดล้อม คณะเกษตรศาสตร์ฯ มหาวิทยาลัยนเรศวร 
ในงานวิจัย เรื่อง "การวิจัยและพัฒนาผลิตภัณฑ์โมเสคฝนประมาณค่าจากเรดาร์ตรวจอากาศในพื้นที่ระดับลุ่มน้ำของประเทศไทยด้วยเทคโนโลยีภูมิสารสนเทศรหัสเปิด"
สนับสนุนทุนวิจัยโดยสํานักงานการวิจัยแห่งชาติ (วช.)  แผนงานการวิจัยและนวัตกรรมแผนงานด้านการบริหารจัดการภัยพิบัติทางธรรมชาติ 
ประจำปีงบประมาณ 2566  ตามสัญญา เลขที่ N25A660467 ผู้นำโค้ดนี้ไปใช้หรือดัดแปลงควรอ้างอิงงานวิจัยชิ้นนี้ตามหลักเกณฑ์การอ้างอิงสากล
เรียนหลักการเรดาร์และภูมิสารสนเทศ ที่ https://www.youtube.com/@Nattapon_Mahavik/playlists
หนังสือเรดาร์ตรวจอากาศทางอุตุนิยมวิทยา สำนักพิมพ์จุฬาฯ : https://www.chulabook.com/education/144567
หนังสือออนไลน์เรดาร์ตรวจอากาศทางอุตุนิยมวิทยา สำนักพิมพ์จุฬาฯ : https://www.chulabook.com/education/205129
ติดต่อ nattaponm@nu.ac.th
'''

# perfect codes
import os
import numpy as np
import pandas as pd
import rasterio
from tqdm import tqdm
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from collections import defaultdict
import warnings

warnings.filterwarnings("ignore", category=RuntimeWarning)

def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

def read_radar_data(file_path):
    with rasterio.open(file_path) as src:
        return src.read(1), src.transform

def extract_radar_rainfall(lon, lat, radar_data, transform):
    row, col = rasterio.transform.rowcol(transform, lon, lat)
    return radar_data[row, col]

def dbz_to_rainfall(dbz, zr_relation):
    Z = 10 ** (dbz / 10)  # Convert dBZ to Z
    
    zr_relations = {
        'marshall-palmer': (200, 1.6),
        'rosenfeld-tropical': (250, 1.2),
        'summer-deep-convective': (300, 1.4)
    }
    
    a, b = zr_relations[zr_relation]
    R = (Z / a) ** (1 / b)
    return R

def calculate_metrics(observed, predicted):
    if len(observed) < 2:
        return {metric: np.nan for metric in ['MSE', 'RMSE', 'R-squared', 'MAE', 'NSE', 'PBIAS', 'Correlation', 'MFB', 'Data_Count']}
    
    mse = mean_squared_error(observed, predicted)
    rmse = np.sqrt(mse)
    r2 = r2_score(observed, predicted)
    mae = mean_absolute_error(observed, predicted)
    
    try:
        nse = 1 - (np.sum((observed - predicted)**2) / np.sum((observed - np.mean(observed))**2))
    except ZeroDivisionError:
        nse = np.nan
    
    try:
        pbias = np.sum(predicted - observed) / np.sum(observed) * 100
    except ZeroDivisionError:
        pbias = np.nan
    
    correlation = np.corrcoef(observed, predicted)[0, 1]
    
    try:
        mfb = np.mean(observed) / np.mean(predicted) # ถ้าค่า mfb > 1 ฝนเรดาร์ประมาณค่าได้น้อยกว่าฝนภาคพื้นดิน
               
    except ZeroDivisionError:
        mfb = np.nan
    
    return {
        'MSE': mse,
        'RMSE': rmse,
        'R-squared': r2,
        'MAE': mae,
        'NSE': nse,
        'PBIAS': pbias,
        'Correlation': correlation,
        'MFB': mfb,
        'Data_Count': len(observed)
    }

def process_data(radar_dir, gauge_dir, match_files_dir, radar_names, thresholds):
    zr_relations = ['marshall-palmer', 'rosenfeld-tropical', 'summer-deep-convective']
    all_results = defaultdict(lambda: defaultdict(lambda: defaultdict(dict)))

    for radar in tqdm(radar_names, desc="Processing radars"):
        radar_specific_dir = os.path.join(radar_dir, f'00outp_cappi_2km_{radar}')
        gauge_specific_dir = os.path.join(gauge_dir, radar, 'hourly')
        match_file_path = os.path.join(match_files_dir, f'matched_files_{radar}.csv')
        
        if not os.path.exists(radar_specific_dir) or not os.path.exists(gauge_specific_dir) or not os.path.exists(match_file_path):
            print(f"Warning: Required directories or match file not found for {radar}")
            continue
        
        match_data = pd.read_csv(match_file_path)
        
        for zr in zr_relations:
            filtered_data_by_threshold = {threshold: [] for threshold in thresholds}
            
            for _, row in tqdm(match_data.iterrows(), desc=f"Processing {radar} files for {zr}", total=len(match_data)):
                radar_file_path = os.path.join(radar_specific_dir, row['Radar_File'])
                gauge_file_path = os.path.join(gauge_specific_dir, row['Gauge_File'])
                
                if not os.path.exists(radar_file_path) or not os.path.exists(gauge_file_path):
                    print(f"Warning: File not found: {row['Radar_File']} or {row['Gauge_File']}")
                    continue
                
                try:
                    radar_data, transform = read_radar_data(radar_file_path)
                    gauge_data = pd.read_csv(gauge_file_path)
                    
                    radar_rainfall = dbz_to_rainfall(radar_data, zr)
                    gauge_data['radar_rainfall'] = gauge_data.apply(
                        lambda x: extract_radar_rainfall(x['longitude'], x['latitude'], radar_rainfall, transform),
                        axis=1
                    )
                    
                    for threshold in thresholds:
                        hourly_filtered = gauge_data[
                            (gauge_data['rain'] > threshold) & 
                            (gauge_data['radar_rainfall'] > threshold)
                        ]
                        if not hourly_filtered.empty:
                            filtered_data_by_threshold[threshold].append(hourly_filtered)
                
                except Exception as e:
                    print(f"Error processing data for {row['Radar_File']}: {str(e)}")
                    continue
            
            for threshold in thresholds:
                if filtered_data_by_threshold[threshold]:
                    aggregated_data = pd.concat(filtered_data_by_threshold[threshold])
                    
                    # Explicitly specify which columns to sum
                    columns_to_sum = ['rain', 'radar_rainfall']
                    
                    event_data = aggregated_data.groupby(['station_code', 'longitude', 'latitude'])[columns_to_sum].sum().reset_index()
                    
                    if len(event_data) > 0:
                        metrics = calculate_metrics(event_data['rain'], event_data['radar_rainfall'])
                    else:
                        metrics = {'Data_Count': 0}
                    
                    all_results[radar][zr][threshold]['full_event'] = metrics
                else:
                    all_results[radar][zr][threshold]['full_event'] = {'Data_Count': 0}

    return all_results

def save_detailed_results(results, output_dir):
    ensure_dir(output_dir)
    
    all_data = []
    for radar in results:
        for zr in results[radar]:
            for threshold in results[radar][zr]:
                metrics = results[radar][zr][threshold]['full_event']
                row = {
                    'Radar': radar,
                    'ZR_Relation': zr,
                    'Threshold': threshold,
                    'Event': 'full_event'
                }
                row.update(metrics)
                all_data.append(row)
    
    df = pd.DataFrame(all_data)
    df.to_csv(os.path.join(output_dir, 'detailed_results.csv'), index=False)

def create_summary(results):
    summary = defaultdict(lambda: defaultdict(lambda: defaultdict(dict)))
    
    for radar in results:
        for zr in results[radar]:
            for threshold in results[radar][zr]:
                summary[radar][zr][threshold] = results[radar][zr][threshold]['full_event']
    
    return summary

def save_summary(summary, output_dir):
    ensure_dir(output_dir)
    
    all_data = []
    for radar in summary:
        for zr in summary[radar]:
            for threshold in summary[radar][zr]:
                row = {
                    'Radar': radar,
                    'ZR_Relation': zr,
                    'Threshold': threshold
                }
                row.update(summary[radar][zr][threshold])
                all_data.append(row)
    
    df = pd.DataFrame(all_data)
    df.to_csv(os.path.join(output_dir, 'summary_results.csv'), index=False)
    
    print("Summary of Results:")
    print(df.to_string())

def main():
    radar_dir = '../00run_batch_acchr_codes/2output/0Hourly/0Sontihn_dbz_single/0processing_single_cappi/'
    gauge_dir = './0Zprocessing_data/0hourly_rain_each_radar/'
    match_files_dir = './0Zprocessing_data/0match_files/'
    output_dir = './0Zprocessing_data/0analysis_results/'
    
    thresholds = [0, 0.1, 0.5, 1, 2, 5, 10]
    radar_names = ['CHN', 'CMP', 'CRI', 'KKN', 'KRB', 'LMP', 'NRT', 'PHS', 'PKT', 'SNK', 'STP', 'SVP']
    #radar_names = ['CHN']

    results = process_data(radar_dir, gauge_dir, match_files_dir, radar_names, thresholds)
    save_detailed_results(results, output_dir)

    summary = create_summary(results)
    save_summary(summary, output_dir)

    print(f"\nDetailed results and summary saved in {output_dir}")

if __name__ == "__main__":
    main()


Processing CHN files for marshall-palmer: 100%|██████████████████████████████████████| 240/240 [00:23<00:00, 10.31it/s]

Processing CHN files for rosenfeld-tropical: 100%|███████████████████████████████████| 240/240 [00:19<00:00, 12.60it/s]

Processing CHN files for summer-deep-convective:   0%|                                         | 0/240 [00:00<?, ?it/s]
Processing CHN files for summer-deep-convective:   1%|▎                                | 2/240 [00:00<00:18, 12.77it/s]
Processing CHN files for summer-deep-convective:   2%|▌                                | 4/240 [00:00<00:18, 12.82it/s]
Processing CHN files for summer-deep-convective:   2%|▊                                | 6/240 [00:00<00:18, 12.87it/s]
Processing CHN files for summer-deep-convective:   3%|█                                | 8/240 [00:00<00:18, 12.62it/s]
Processing CHN files for summer-deep-convective:   4%|█▎                              | 10/240 [00:00<00:18, 12.67it/s]
Processing CHN files for summer-deep-c

Summary of Results:
    Radar             ZR_Relation  Threshold          MSE       RMSE   R-squared        MAE         NSE       PBIAS  Correlation       MFB  Data_Count
0     CHN         marshall-palmer        0.0  1058.220329  32.530299   -1.338897  16.289322   -1.338897   91.661884     0.543103  0.521752       110.0
1     CHN         marshall-palmer        0.1  1051.375498  32.424921   -1.323847  16.170170   -1.323847   90.888228     0.544583  0.523867       110.0
2     CHN         marshall-palmer        0.5   472.322465  21.732981   -0.003686  11.573633   -0.003686   27.794158     0.601690  0.782508        88.0
3     CHN         marshall-palmer        1.0   184.255411  13.574071    0.568714   8.280277    0.568714   -2.689325     0.754652  1.027636        67.0
4     CHN         marshall-palmer        2.0   133.925617  11.572624    0.631862   6.776281    0.631862  -13.179270     0.817431  1.151799        47.0
5     CHN         marshall-palmer        5.0    57.867555   7.607073    0.

## เขียนค่าสถิติตาม threshold เพื่อนำไปใช้ต่อในการโมเสคด้วยการใช้ MFB เป็นตัวคูณปรับแก้ก่อนโมเสค

In [ ]:
def process_summary_results(input_file, output_dir):
    # Read the summary results
    df = pd.read_csv(input_file)
    
    # เลือกสร้างไฟล์ที่มีค่าฝนมากกว่า 0 และ 1.0 ในระดับรายชั่วโมง
    thresholds = [0, 1.0]
    
    for threshold in thresholds:
        # Filter the dataframe for the current threshold
        df_threshold = df[df['Threshold'] == threshold]
        
        # Prepare the output dataframe, now including the Threshold column
        output_df = df_threshold[['Radar', 'ZR_Relation', 'Threshold', 'MSE', 'RMSE', 'R-squared', 'MAE', 'NSE', 'PBIAS', 'Correlation', 'MFB']]
        
        # Sort the dataframe by Radar and ZR_Relation
        output_df = output_df.sort_values(['Radar', 'ZR_Relation'])
        
        # Define the output file path
        output_file = os.path.join(output_dir, f'combined_results_threshold_{threshold}.csv')
        
        # Save the dataframe to CSV
        output_df.to_csv(output_file, index=False)
        
        print(f"File saved: {output_file}")

# Usage
input_file = './0Zprocessing_data/0analysis_results/summary_results.csv'
output_dir = './0Zprocessing_data/0analysis_results/'

process_summary_results(input_file, output_dir)
